In [1]:
import time
start_time = time.time()

# Preparation - Data grouped on forum level

Import the packages and load the data

In [2]:
# Import needed packages
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import KBinsDiscretizer
import seaborn as sns; sns.set()
import glob
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import re
import nltk
from langdetect import detect
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from lime import lime_text
import gensim
import gensim.downloader as gensim_api
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
import transformers
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy
from spacy import displacy
from wordcloud import WordCloud
from sklearn.preprocessing import KBinsDiscretizer
from nbpep8.nbpep8 import pep8
from sklearn.metrics.pairwise import cosine_similarity
from linearmodels import PanelOLS


In [3]:
#Set the working directory
os.chdir('C:\\Users\\piotr\\OneDrive - Erasmus University Rotterdam\\Thesis\\')

In [4]:
# #Load the merged_df from the directory 'C:\\Users\\piotr\\OneDrive - Erasmus University Rotterdam\\Thesis\\Master Thesis Data\\' (containing the readability results)):
merged_df = pd.read_csv('Master Thesis Data\\df_clean_final.csv')

In [5]:
#Load the results of the content novelty calculation, that were saved as follows:
final_result_df_tfidf = pd.read_csv('Master Thesis Data\\final_result_df_tfidf.csv')
final_result_df_lsi = pd.read_csv('Master Thesis Data\\final_result_df_lsi.csv')
final_result_df_doc2vec = pd.read_csv('Master Thesis Data\\final_result_df_doc2vec.csv')

Load the grouped data frames

In [6]:
#Read the grouped dfs, exclude loading 'rix' and 'lix' columns
grouped_df = pd.read_csv('Master Thesis Data\\grouped_df.csv')
grouped_df_tag = pd.read_csv('Master Thesis Data\\grouped_df_tag.csv')
grouped_df_week = pd.read_csv('Master Thesis Data\\grouped_df_week.csv')
grouped_df_week_tag = pd.read_csv('Master Thesis Data\\grouped_df_week_tag.csv')

Drop rix and lix

In [7]:
#Drop rix and lix
merged_df = merged_df.drop(['rix', 'lix'], axis=1)
grouped_df = grouped_df.drop(['rix', 'lix'], axis=1)
grouped_df_tag = grouped_df_tag.drop(['rix', 'lix'], axis=1)
grouped_df_week = grouped_df_week.drop(['rix', 'lix'], axis=1)
grouped_df_week_tag = grouped_df_week_tag.drop(['rix', 'lix'], axis=1)


Drop text fields

In [8]:
# merged_df drop Post Body, Post Body CLean, Post Body EN and Post title
merged_df = merged_df.drop(['Post_Body', 'Post_Body_clean', 'Post_Body_EN', 'Post_Title'], axis=1)

# 7. Estimation

Assume that we have grouped_df (grouped by month). Month indicator is 'Month_of_Year'.
Dependent variables are:
1.Post_Count by forum
2.Post_Answer_Count by forum
3.Avg_readability_index by forum
4.avg_cos_sim by tag

For the DV 1-3 it is possible to test the PTA assumption by adding time dummies to regression. 
The content novelty has got only two periods per group, so it cannot be tested that way.


## Preparation - part 2

#### Needed transformations for merged_df, grouped_df and grouped_df_tag 

In [9]:
merged_df['Post_Creation_Date'] = pd.to_datetime(merged_df['Post_Creation_Date'])
merged_df['Month_of_Year'] = merged_df['Post_Creation_Date'].dt.month

In [10]:
#Add the variable 'Month counter' to the merged_df, taking value -4 for 07 -3 for 08 -2 for 09 -1 for 10 0 for 11 1 for 12 2 for 01 3 for 02 4 for 03 5 for 04 #use map
merged_df['Month_Counter'] = merged_df['Month_of_Year'].map({7:-4, 8:-3, 9:-2, 10:-1, 11:0, 12:1, 1:2, 2:3, 3:4, 4:5})

In [11]:
#display all cols
pd.set_option('display.max_columns', None)

In [12]:
merged_df_2 = merged_df.copy()

In [13]:
grouped_df_2 = grouped_df.copy()
grouped_df_tag_2 = grouped_df_tag.copy()
grouped_df_week_2 = grouped_df_week.copy()
grouped_df_week_tag_2 = grouped_df_week_tag.copy()

## Limit the number of tags-combined in the weekly data frames

In [14]:
#Show tag combined in merged_df that have more than 1000 observations
merged_df['tag_combined'].value_counts()[merged_df['tag_combined'].value_counts() > 500]

pandaspython          54032
javascriptreactjs     47767
htmljavascript        46838
pythonpython-3.x      34593
csshtml               31791
                      ...  
c#string                504
rraster                 504
flutterflutter-web      503
jsonswift               502
scala                   502
Name: tag_combined, Length: 581, dtype: int64

In [15]:
merged_df.head()

,Post_Creation_Date,Post_View_Count,Post_Score,Post_Answer_Count,Response_Scores,Post_Author,Answer_Author,Forum,Week_of_Year,tag_1,tag_2,tag_3,tag_combined,Weekly_Post_Count,D,T,Title_Length,Body_Length,ari,fre,fk_grade,gunning_fog,smog_index,coleman_liau_index,Month_of_Year,Month_Counter
0,2021-07-01 00:00:58,5867,1,3,1,8530236,8530236,Stack Overflow,26,ruby-on-rails,ssl,NaN,ruby-on-railsssl,30616,0,0,59,315,6.1,72.83,4.8,8.51,8.8,8.01,7,-4
1,2021-07-01 00:01:10,11,0,0,0,3893947,0,Stack Overflow,26,ms-word,NaN,NaN,ms-word,30616,0,0,65,718,7.3,79.60,6.4,7.81,8.4,6.44,7,-4
2,2021-07-01 00:01:18,1393,1,1,2,2313889,13107804,Stack Overflow,26,python,plotly,subplot,plotlypython,30616,0,0,48,1095,11.4,57.61,10.7,14.25,0.0,9.34,7,-4
3,2021-07-01 00:01:28,660,1,2,1,15301811,4014834,Stack Overflow,26,javascript,html,d3.js,htmljavascript,30616,0,0,41,7146,5.0,78.25,4.8,6.00,8.5,6.06,7,-4
4,2021-07-01 00:02:36,59,0,2,2,14595783,1440565,Stack Overflow,26,java,android,xml,androidjava,30616,0,0,41,12859,10.3,66.27,9.4,10.05,10.2,7.83,7,-4


In [16]:
# Ensure 'Post_Creation_Date' is of datetime type
merged_df['Post_Creation_Date'] = pd.to_datetime(merged_df['Post_Creation_Date'])

# Create a DataFrame that shows the weekly counts for each 'tag_combined'
weekly_counts = merged_df.groupby([pd.Grouper(key='Post_Creation_Date', freq='W'), 'tag_combined']).size().unstack().fillna(0)

# Find 'tag_combined' that have at least 1 observation in each week
tags_with_obs_each_week = weekly_counts.loc[:, (weekly_counts > 0).all()]

# Get a list of these tags
tags_list = tags_with_obs_each_week.columns.tolist()

# Recode tag combinations in merged_df that don't meet the criteria
merged_df['tag_combined'] = merged_df['tag_combined'].apply(lambda x: x if x in tags_list else 'other')


In [17]:
#Show the 5 most popular tag combined
merged_df['tag_combined'].value_counts().head(5)

other                1676773
pandaspython           54032
javascriptreactjs      47767
htmljavascript         46838
pythonpython-3.x       34593
Name: tag_combined, dtype: int64

In [18]:
#drop other tags
merged_df_reduced = merged_df[merged_df['tag_combined'] != 'other']


In [19]:
grouped_df_tag_reduced_week = merged_df_reduced.groupby(['tag_combined', merged_df_reduced['Post_Creation_Date'].dt.to_period('W')]).agg(
    Post_Count=('Post_Creation_Date', 'count'),
    Post_Answer_Count=('Post_Answer_Count', 'sum'),
    ari = ('ari', 'mean'),
    fre = ('fre', 'mean'),
    fk_grade = ('fk_grade', 'mean'),
    gunning_fog = ('gunning_fog', 'mean'),
    D = ('D', 'mean'),
    T = ('T', 'mean'),
    smog_index = ('smog_index', 'mean'),
    coleman_liau_index = ('coleman_liau_index', 'mean'),
    Week_of_Year=('Post_Creation_Date', lambda x: x.dt.isocalendar().week.min()),
    Post_View_Count=('Post_View_Count', 'sum'),
    Post_Score=('Post_Score', 'sum'),
    Response_Scores=('Response_Scores', 'sum'),
    Post_Author=('Post_Author', 'nunique'),
    Answer_Author=('Answer_Author', 'nunique'),
    Title_Length = ('Title_Length', 'mean'),
    Body_Length=('Body_Length', 'mean')
).reset_index()


In [20]:
grouped_df_tag_reduced_week

,tag_combined,Post_Creation_Date,Post_Count,Post_Answer_Count,ari,fre,fk_grade,gunning_fog,D,T,smog_index,coleman_liau_index,Week_of_Year,Post_View_Count,Post_Score,Response_Scores,Post_Author,Answer_Author,Title_Length,Body_Length
0,.htaccessapache,2021-06-28/2021-07-04,9,8,9.066667,70.634444,7.066667,8.372222,0.0,0.0,6.944444,8.450000,26,708,7,7,7,5,48.777778,1095.555556
1,.htaccessapache,2021-07-05/2021-07-11,8,6,9.875000,67.343750,8.762500,10.328750,0.0,0.0,9.062500,7.348750,27,1187,11,9,8,3,55.250000,1004.500000
2,.htaccessapache,2021-07-12/2021-07-18,6,5,9.833333,65.216667,7.750000,7.706667,0.0,0.0,9.050000,9.850000,28,2590,7,12,6,4,66.000000,1780.166667
3,.htaccessapache,2021-07-19/2021-07-25,5,4,10.720000,61.534000,8.760000,9.546000,0.0,0.0,10.780000,9.900000,29,1115,5,4,5,5,73.000000,2025.200000
4,.htaccessapache,2021-07-26/2021-08-01,8,7,8.075000,70.766250,6.675000,7.420000,0.0,0.0,7.587500,8.008750,30,1427,7,14,8,2,56.000000,1571.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49451,xmlxslt,2023-03-27/2023-04-02,9,9,9.977778,64.838889,8.144444,9.123333,1.0,1.0,10.277778,9.716667,13,340,1,7,8,4,56.555556,3510.000000
49452,xmlxslt,2023-04-03/2023-04-09,7,9,15.357143,61.958571,12.571429,14.960000,1.0,1.0,2.928571,7.481429,14,284,-1,3,7,4,72.857143,4407.571429
49453,xmlxslt,2023-04-10/2023-04-16,1,1,14.500000,50.910000,13.300000,15.110000,1.0,1.0,14.400000,9.300000,15,59,0,1,1,1,57.000000,3221.000000
49454,xmlxslt,2023-04-17/2023-04-23,7,5,4.842857,112.841429,-0.200000,5.054286,1.0,1.0,6.428571,0.718571,16,212,1,5,7,4,66.428571,2195.285714


In [21]:
grouped_df_week_tag_reduced = grouped_df_tag_reduced_week.copy()

In [22]:
# #Save grouped_df_tag_reduced as csv
# grouped_df_week_tag_reduced.to_csv('Master Thesis Data\\grouped_df_week_tag_reduced.csv', index=False)

In [23]:
print('merged_df' , merged_df.columns, merged_df.shape) 
print(grouped_df.columns, grouped_df.shape)
print(grouped_df_tag.columns, grouped_df_tag.shape)
print(grouped_df_week.columns, grouped_df_week.shape)
print(grouped_df_week_tag.columns, grouped_df_week_tag.shape)
print(grouped_df_tag_reduced_week.columns, grouped_df_tag_reduced_week.shape)


merged_df Index(['Post_Creation_Date', 'Post_View_Count', 'Post_Score',
       'Post_Answer_Count', 'Response_Scores', 'Post_Author', 'Answer_Author',
       'Forum', 'Week_of_Year', 'tag_1', 'tag_2', 'tag_3', 'tag_combined',
       'Weekly_Post_Count', 'D', 'T', 'Title_Length', 'Body_Length', 'ari',
       'fre', 'fk_grade', 'gunning_fog', 'smog_index', 'coleman_liau_index',
       'Month_of_Year', 'Month_Counter'],
      dtype='object') (3040730, 26)
Index(['Forum', 'Post_Creation_Date', 'Post_Count', 'Post_Answer_Count', 'ari',
       'fre', 'fk_grade', 'gunning_fog', 'smog_index', 'coleman_liau_index',
       'Week_of_Year', 'Month_of_Year', 'Post_View_Count', 'Post_Score',
       'Response_Scores', 'Post_Author', 'Answer_Author', 'Title_Length',
       'Body_Length', 'Month_Counter', 'D', 'T'],
      dtype='object') (21199, 22)
Index(['tag_combined', 'Post_Creation_Date', 'Post_Count', 'Post_Answer_Count',
       'ari', 'fre', 'fk_grade', 'gunning_fog', 'smog_index',
       'colem

In [24]:
# Create a dictionary of dataframes
dfs_est = {
    "grouped_df_week": grouped_df_week.copy(),
    "grouped_df_week_tag": grouped_df_week_tag.copy(),
}

# Your function to modify a dataframe
def modify_dataframe(df):
      # Extract the start date of each week and convert it to datetime
    df['Week_Start_Date'] = df['Post_Creation_Date'].apply(lambda x: datetime.strptime(x.split('/')[0], '%Y-%m-%d'))

    # Find the minimum date in the DataFrame
    min_date = df['Week_Start_Date'].min()

    # Create a new column 'Week_Number' that calculates the difference in weeks from the minimum date
    df['Week_Number'] = (df['Week_Start_Date'] - min_date).dt.days // 7 + 1

    # Add the interaction term
    df['DiD'] = df['D'] * df['T']
    
    return df

# # Apply the function to each dataframe and save as CSV
# for df_name, df in dfs_est.items():
#     modified_df = modify_dataframe(df.copy())
#     modified_df.to_csv(f'Master Thesis Data\\estimation_dfs\\est_{df_name}.csv', index=False)


In [25]:
import pandas as pd
from datetime import datetime

# assuming df is your DataFrame
df = grouped_df_week_tag_reduced.copy()

# Extract the start date of each week and convert it to datetime
df['Week_Start_Date'] = df['Post_Creation_Date'].apply(lambda x: x.start_time)

# Find the minimum date in the DataFrame
min_date = df['Week_Start_Date'].min()

# Create a new column 'Week_Number' that calculates the difference in weeks from the minimum date
df['Week_Number'] = (df['Week_Start_Date'] - min_date).dt.days // 7 + 1

# Add the interaction term
df['DiD'] = df['D'] * df['T']

print(df.head())

# #Save as csv
# df.to_csv('Master Thesis Data\\estimation_dfs\\est_grouped_df_week_tag_reduced.csv', index=False)

      tag_combined     Post_Creation_Date  Post_Count  Post_Answer_Count  \
0  .htaccessapache  2021-06-28/2021-07-04           9                  8   
1  .htaccessapache  2021-07-05/2021-07-11           8                  6   
2  .htaccessapache  2021-07-12/2021-07-18           6                  5   
3  .htaccessapache  2021-07-19/2021-07-25           5                  4   
4  .htaccessapache  2021-07-26/2021-08-01           8                  7   

         ari        fre  fk_grade  gunning_fog    D    T  smog_index  \
0   9.066667  70.634444  7.066667     8.372222  0.0  0.0    6.944444   
1   9.875000  67.343750  8.762500    10.328750  0.0  0.0    9.062500   
2   9.833333  65.216667  7.750000     7.706667  0.0  0.0    9.050000   
3  10.720000  61.534000  8.760000     9.546000  0.0  0.0   10.780000   
4   8.075000  70.766250  6.675000     7.420000  0.0  0.0    7.587500   

   coleman_liau_index  Week_of_Year  Post_View_Count  Post_Score  \
0             8.45000            26       

# EDA from here